In [1]:
from __future__ import print_function
import os
import neat

import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
# from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter
from explaneat.core.utility import one_hot_encode


from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from copy import deepcopy

import time
from datetime import datetime


import gzip
try:
    import cPickle as pickle  # pylint: disable=import-error
except ImportError:
    import pickle  # pylint: disable=import-error

In [2]:

USE_CUDA = torch.cuda.is_available()
# USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")


# Breast Cancer Experiment

This experiment (a) test the experimental environment, but is also to evaluate the efficacy of the ExplaNEAT algorithm. Speed is a critical factor, as well as stability of results on population size. Total run time will also be measured

First, we need to set a random seed and a total stopping point in the number of generations

In [3]:
my_random_seed = 42
random.seed(my_random_seed)

In [4]:
def one_hot_encode(vals):
    width = max(vals)
    newVals = []
    for val in vals:
        blank = [0. for _ in range(width + 1)]
        blank[val] = 1.
        newVals.append(blank)
    return np.asarray(newVals)


## Dataset

We are going to work with the Iris dataset, which will be loaded from `sklearn`. We want to characterise the efficacy of the algorithm with regards to a mostly untransformed dataset, so we will only normalise the features

In [5]:
breast_cancer = datasets.load_breast_cancer()
xs_raw = breast_cancer.data[:, :]
scaler = StandardScaler()
scaler.fit(xs_raw)
xs = scaler.transform(xs_raw)
ys = breast_cancer.target.astype(np.float32)
# ys_onehot = one_hot_encode(ys)

In [6]:
xs = torch.from_numpy(xs).to(device)
ys = torch.from_numpy(ys).to(device)

Let's have a look at the data we are working with

In [7]:
ys[18:23]

tensor([0., 1., 1., 1., 0.])

In [8]:
xs[18:20]

tensor([[ 1.6140,  0.6656,  1.5665,  1.7210,  0.1388, -0.0311,  0.7420,  1.1881,
         -0.8383, -1.2542,  1.2742, -0.3626,  1.4846,  1.5855, -0.1823, -0.3660,
          0.0669,  0.5538, -0.8454, -0.6801,  2.2884,  0.8472,  2.3691,  2.6675,
          0.8255,  0.3864,  1.2714,  1.8910, -0.2148, -0.4320],
        [-0.1668, -1.1472, -0.1857, -0.2520,  0.1017, -0.4369, -0.2782, -0.0286,
          0.2679, -0.7283, -0.4882, -0.7770, -0.4000, -0.3691,  0.4737, -0.6080,
         -0.2660,  0.2196, -0.0899, -0.5654, -0.2400, -1.0450, -0.2252, -0.2978,
          0.5099, -0.4896, -0.1592,  0.2161,  0.1233, -0.6293]],
       dtype=torch.float64)

## Performance metric

The NEAT implementation on which ExplaNEAT extends uses a single function call for evaluating fitness. Although this might be reworked for ExplaNEAT to be able to get consistency between the genome-evaluation and the backprop loss function, that can be reviewed later.

This use `Binary Cross Entropy Loss` from `PyTorch`

In [9]:
def eval_genomes(genomes, config):
    loss = nn.BCELoss()
    loss = loss.to(device)
    for genome_id, genome in genomes.items():
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in xs:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(ys)))

## Base configuration

We are going to create the base configuration according to an external configuration file. Per experiment, we will adjust this, later, but this defines the defaults across all runs.

In [10]:
config_path = "./config-breast-cancer"
base_config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)


In [11]:
base_config.pop_size

2

We also want to put a hard limit on how long this can go on for.

In [12]:
maxNGenerations = 20

We will create a method to manage the instantiation of a population on the basis of a specific config.

In [13]:
def instantiate_population(config, xs, ys, saveLocation):

    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
        
    config.save(os.path.join(saveLocation, 'config.conf'))

    # Create the population, which is the top-level object for a NEAT run.
    p = BackpropPopulation(config, 
                            xs, 
                            ys, 
                            criterion=nn.BCELoss())

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5, filename_prefix=str(saveLocation) + "checkpoint-" ))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)
    p.add_reporter(ExperimentReporter(saveLocation))
    
    return p

# Experiment 1: Vary population size

The first experiment is going to examine the difference in run time different population sizes. 

In [14]:
population_points = [2, 5, 10, 25, 50, 100]

In [15]:
base_config.pop_size

2

In [16]:
saveLocationTemplate = './../../data/experiments/breast-cancer/experiment-populationgpu-{}-{}/'

## Start the experiment

In [ ]:
for pop_size in population_points:
    for iteration_no in range(5):
        my_random_seed += 1
        random.seed(my_random_seed)
        start_time = datetime.now()
        
        print("################################################")
        print("################################################")
        print("Starting population {} iteration {}".format(pop_size, iteration_no))
        print("Started at {}".format(start_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("################################################")
        print("################################################")
        
        
        config = deepcopy(base_config)
        config.pop_size = pop_size
        
        saveLocation = saveLocationTemplate.format(pop_size, iteration_no)
        
        p = instantiate_population(config, xs, ys, saveLocation)
        # Run for up to nGenerations generations.
        winner = p.run(eval_genomes, maxNGenerations)
        
        g = p.best_genome

        
        end_time = datetime.now()
        
        p.reporters.reporters[2].save_checkpoint(p.config, p.population, p.species, str(p.generation) + "-final")  
        
        winner_net = neat.nn.FeedForwardNetwork.create(winner, config)

        results = []
        for xi, xo in zip(xs, ys):
            output = winner_net.activate(xi)
            results.append([xi, xo, output])

        df = pd.DataFrame(results)
        df.to_csv(os.path.join(saveLocation, 'results.csv'))
        
        ancestry = p.reporters.reporters[3].trace_ancestry_of_species(g.key, p.reproduction.ancestors) 

        ancestors = {
            k: v['genome'] for k, v in p.reporters.reporters[3].ancestry.items()
        }
    
        
        ## Save all of these to disc
        filename = 'fullStatus.xplnt'
        print("Saving checkpoint to {0}".format(filename))

        with gzip.open(os.path.join(saveLocation, filename), 'w', compresslevel=5) as f:
            data = (p, g, ancestry, ancestors, random.getstate())
            pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
#         visualize.create_ancestry_video(p.config, 
#                                         g, 
#                                         ancestry, 
#                                         ancestors, 
#                                         p.reporters.reporters[1], 
#                                         pathname=saveLocation)
        print("################################################")
        print("################################################")
        print("Have finished population {} iteration {}".format(pop_size, iteration_no))
        print("Started at {}".format(start_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("The time is {}".format(end_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("################################################")
        print("################################################")
    

################################################
################################################
Starting population 2 iteration 0
Started at 07/06/2020, 09:40:02
################################################
################################################
The function - generationStart - has just started at 1593985202.76916

 ****** Running generation 0 ****** 

The function - generationStart - took 0.00031304359436035156 seconds to complete
The function - pre_backprop - has just started at 1593985202.769495
The function - pre_backprop - took 4.792213439941406e-05 seconds to complete
The function - backprop - has just started at 1593985202.7695649
mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(3.8294, grad_fn=<DivBackward0>)
The function - backprop - took 126.71428418159485 seconds to complete
The function - post_backprop - has just started at 1593985329.484215
The function - post_backprop - took 0.0003981590270996094 seconds to compl

<ipython-input-9-b299ef3b426e>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(ys)))
/Users/mike/opt/anaconda3/envs/phd-neat/lib/python3.8/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([569])) that is different to the input size (torch.Size([569, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


The function - evaluate fitness - took 0.5072498321533203 seconds to complete
The function - post evaluate - has just started at 1593985329.992855
Population's average fitness: 0.24651 stdev: 0.01864
Best fitness: 0.26515 - size: (1, 30) - species 1 - id 1
Key: 1
Fitness: 0.26514601707458496
Nodes:
	0 DefaultNodeGene(key=0, bias=1.4986084699630737, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-30, 0), weight=-0.7101625204086304, enabled=True)
	DefaultConnectionGene(key=(-29, 0), weight=0.4450584948062897, enabled=True)
	DefaultConnectionGene(key=(-28, 0), weight=0.8950620889663696, enabled=True)
	DefaultConnectionGene(key=(-27, 0), weight=2.2113723754882812, enabled=True)
	DefaultConnectionGene(key=(-26, 0), weight=0.2212318331003189, enabled=True)
	DefaultConnectionGene(key=(-25, 0), weight=-0.22664980590343475, enabled=True)
	DefaultConnectionGene(key=(-24, 0), weight=-0.05486959218978882, enabled=True)
	DefaultConnectionGene(key=(-23, 0

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(3.8294, grad_fn=<DivBackward0>)
The function - backprop - took 138.87920308113098 seconds to complete
The function - post_backprop - has just started at 1593985606.553824
The function - post_backprop - took 9.799003601074219e-05 seconds to complete
The function - evaluate fitness - has just started at 1593985606.553965
The function - evaluate fitness - took 0.46303486824035645 seconds to complete
The function - post evaluate - has just started at 1593985607.017322
Population's average fitness: 0.24651 stdev: 0.01864
Best fitness: 0.26515 - size: (1, 30) - species 1 - id 1
Key: 1
Fitness: 0.26514601707458496
Nodes:
	0 DefaultNodeGene(key=0, bias=1.4986084699630737, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-30, 0), weight=-0.7101625204086304, enabled=True)
	DefaultConnectionGene(key=(-29, 0), weight=0.4450584948062897, enabled=True)
	DefaultConne

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(3.8294, grad_fn=<DivBackward0>)
The function - backprop - took 103.3874409198761 seconds to complete
The function - post_backprop - has just started at 1593985813.2212322
The function - post_backprop - took 4.792213439941406e-05 seconds to complete
The function - evaluate fitness - has just started at 1593985813.221361
The function - evaluate fitness - took 0.42005491256713867 seconds to complete
The function - post evaluate - has just started at 1593985813.641523
Population's average fitness: 0.24651 stdev: 0.01864
Best fitness: 0.26515 - size: (1, 30) - species 1 - id 1
Key: 1
Fitness: 0.26514601707458496
Nodes:
	0 DefaultNodeGene(key=0, bias=1.4986084699630737, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-30, 0), weight=-0.7101625204086304, enabled=True)
	DefaultConnectionGene(key=(-29, 0), weight=0.4450584948062897, enabled=True)
	DefaultConne

...

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(5.2585, grad_fn=<DivBackward0>)
The function - backprop - took 369.1462080478668 seconds to complete
The function - post_backprop - has just started at 1593990769.555601
The function - post_backprop - took 0.0007059574127197266 seconds to complete
The function - evaluate fitness - has just started at 1593990769.5567
The function - evaluate fitness - took 5.116073846817017 seconds to complete
The function - post evaluate - has just started at 1593990774.673056
Population's average fitness: 0.17437 stdev: 0.01745
Best fitness: 0.19351 - size: (1, 29) - species 2 - id 4
Key: 4
Fitness: 0.19350585341453552
Nodes:
	0 DefaultNodeGene(key=0, bias=0.004062399733811617, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-30, 0), weight=0.46158745884895325, enabled=True)
	DefaultConnectionGene(key=(-29, 0), weight=0.15304073691368103, enabled=True)
	DefaultConnect

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(5.2585, grad_fn=<DivBackward0>)
The function - backprop - took 316.5135130882263 seconds to complete
The function - post_backprop - has just started at 1593991471.70846
The function - post_backprop - took 5.698204040527344e-05 seconds to complete
The function - evaluate fitness - has just started at 1593991471.708559
The function - evaluate fitness - took 1.187474012374878 seconds to complete
The function - post evaluate - has just started at 1593991472.89664
Population's average fitness: 0.17437 stdev: 0.01745
Best fitness: 0.19351 - size: (1, 29) - species 2 - id 4
Key: 4
Fitness: 0.19350585341453552
Nodes:
	0 DefaultNodeGene(key=0, bias=0.004062399733811617, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-30, 0), weight=0.46158745884895325, enabled=True)
	DefaultConnectionGene(key=(-29, 0), weight=0.15304073691368103, enabled=True)
	DefaultConnect

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(5.2585, grad_fn=<DivBackward0>)
The function - backprop - took 336.6383879184723 seconds to complete
The function - post_backprop - has just started at 1593992118.452498
The function - post_backprop - took 0.00035500526428222656 seconds to complete
The function - evaluate fitness - has just started at 1593992118.453119
The function - evaluate fitness - took 1.4995100498199463 seconds to complete
The function - post evaluate - has just started at 1593992119.952758
Population's average fitness: 0.17437 stdev: 0.01745
Best fitness: 0.19351 - size: (1, 29) - species 2 - id 4
Key: 4
Fitness: 0.19350585341453552
Nodes:
	0 DefaultNodeGene(key=0, bias=0.004062399733811617, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-30, 0), weight=0.46158745884895325, enabled=True)
	DefaultConnectionGene(key=(-29, 0), weight=0.15304073691368103, enabled=True)
	DefaultCon

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(5.2585, grad_fn=<DivBackward0>)
The function - backprop - took 335.6353793144226 seconds to complete
The function - post_backprop - has just started at 1593992890.6948018
The function - post_backprop - took 5.030632019042969e-05 seconds to complete
The function - evaluate fitness - has just started at 1593992890.694897
The function - evaluate fitness - took 1.391451120376587 seconds to complete
The function - post evaluate - has just started at 1593992892.087421
Population's average fitness: 0.17437 stdev: 0.01745
Best fitness: 0.19351 - size: (1, 29) - species 2 - id 4
Key: 4
Fitness: 0.19350585341453552
Nodes:
	0 DefaultNodeGene(key=0, bias=0.004062399733811617, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-30, 0), weight=0.46158745884895325, enabled=True)
	DefaultConnectionGene(key=(-29, 0), weight=0.15304073691368103, enabled=True)
	DefaultConn

gen is 19
previous generation is [4]
skey is 4
gen is 18
previous generation is [4]
skey is 4
gen is 17
previous generation is [4]
skey is 4
gen is 16
previous generation is [4]
skey is 4
gen is 15
previous generation is [4]
skey is 4
gen is 14
previous generation is [4]
skey is 4
gen is 13
previous generation is [4]
skey is 4
gen is 12
previous generation is [4]
skey is 4
gen is 11
previous generation is [4]
skey is 4
gen is 10
previous generation is [4]
skey is 4
gen is 9
previous generation is [4]
skey is 4
gen is 8
previous generation is [4]
skey is 4
gen is 7
previous generation is [4]
skey is 4
gen is 6
previous generation is [4]
skey is 4
gen is 5
previous generation is [4]
skey is 4
gen is 4
previous generation is [4]
skey is 4
gen is 3
previous generation is [4]
skey is 4
gen is 2
previous generation is [4]
skey is 4
gen is 1
previous generation is [4]
skey is 4
gen is 0
previous generation is [2, 4]
skey is 2
skey is 4
have calculated the ancestry
Saving checkpoint to fullSta

In [ ]:
for indiv in iter(p.population):
    print(indiv)

In [ ]:
for a, b, in p.population.items():
    print(a, b)
    

In [ ]:
p